In [5]:
from const import *
import time
from web3 import Web3
from os import path
import json


def register_user(usrnm, wallet, rfrr):
    nonce = web3.eth.getTransactionCount(OWNER)
    tx = contract.functions.registerUser(usrnm, web3.toChecksumAddress(wallet), rfrr).buildTransaction({
        'chainId': CHAIN_ID, 'nonce': nonce, 'gas': 500000, 'gasPrice': 10000000000,
    })
    sign_tx = web3.eth.account.signTransaction(tx, PRIV_KEY)
    try:
        web3.eth.sendRawTransaction(sign_tx.rawTransaction)
        print(f"Register User successful for {usrnm}, {wallet}, {rfrr}")
        return True
    except ValueError:
        print(f"Register User failed for {usrnm}, {wallet}, {rfrr}")
        f = open(FAILED_FILE, "a")
        f.write(f"Register User: {usrnm}, {wallet}, {rfrr}\n")
        f.close()
        return False


def register_tickets(usrnm, tkns1, tkns2, tkns3):
    nonce = web3.eth.getTransactionCount(OWNER)
    tx = contract.functions.registerTickets(usrnm, tkns1, tkns2, tkns3).buildTransaction({
        'chainId': CHAIN_ID, 'nonce': nonce, 'gas': 500000, 'gasPrice': 10000000000,
    })
    sign_tx = web3.eth.account.signTransaction(tx, PRIV_KEY)
    try:
        web3.eth.sendRawTransaction(sign_tx.rawTransaction)
        print(f"Register Tickets successful for {usrnm}, {tkns1}, {tkns2}, {tkns3}")
        return True
    except ValueError:
        print(f"Register Tickets failed for {usrnm}, {tkns1}, {tkns2}, {tkns3}")
        f = open(FAILED_FILE, "a")
        f.write(f"Register Tickets: {usrnm}, {tkns1}, {tkns2}, {tkns3}\n")
        f.close()
        return False

In [6]:
web3 = Web3(Web3.HTTPProvider(BSC))
contract = web3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

all_users = {}
all_tickets = {}

if path.exists("../"+USER_FILE):
    f = open("../"+USER_FILE, "r")
    all_users = json.load(f)
    f.close()

if path.exists("../"+TICKET_FILE):
    f = open("../"+TICKET_FILE, "r")
    all_tickets = json.load(f)
    f.close()

In [7]:
for k, v in all_users.items():
    if v[2] == 0:
        res = register_user(k, v[0], v[1])
        if res:
            v[2] = 1
            time.sleep(20)
f = open("../"+USER_FILE, "w")
f.write(json.dumps(all_users))
f.close()

Register User successful for 91199062, 0xd8549cef6d354e40cfa5cab236d44a26355c2bce, 
Register User successful for 658843500, 0xd6E0fa4B052A38efA5C383A2997b6b6842BD3F75, 
Register User successful for 138086094, 0xD0f3F5a7684d329a03cadf266f0173b64a143179, 


In [11]:
for k, v in all_tickets.items():
    n = len(v)
    for i in range(n//3):
        if v[3*i][1] == 0 and v[3*i+1][1] == 0  and v[3*i+2][1] == 0:
            res = register_tickets(k, v[3*i][0], v[3*i+1][0], v[3*i+2][0])
            if res:
                v[3*i][1] = 1
                v[3*i+1][1] = 1
                v[3*i+2][1] = 1
                time.sleep(20)
f = open("../"+TICKET_FILE, "w")
f.write(json.dumps(all_tickets))
f.close()

Register Tickets successful for 221514152, [1, 2, 3, 4, 5, 6], [11, 12, 13, 14, 15, 16], [31, 32, 33, 34, 35, 36]
Register Tickets successful for 91199062, [35, 29, 46, 22, 16, 1], [11, 25, 33, 39, 47, 43], [1, 3, 11, 13, 21, 23]
Register Tickets successful for 658843500, [23, 4, 28, 22, 8, 47], [6, 32, 20, 39, 24, 3], [25, 26, 34, 39, 27, 6]
Register Tickets successful for 138086094, [8, 22, 26, 12, 36, 42], [6, 2, 13, 21, 18, 9], [34, 14, 31, 17, 41, 7]
Register Tickets failed for 77287999, [7, 8, 9, 10, 11, 12], [17, 18, 19, 20, 21, 22], [37, 38, 39, 40, 41, 42]


FileNotFoundError: [Errno 2] No such file or directory: 'TDB/failed_file.txt'